In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing


In [2]:
! pip install mlflow

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.7 MB 2.8 MB/s eta 0:00:09
    --------------------------------------- 0.3/24.7 MB 3.3 MB/s eta 0:00:08
    --------------------------------------- 0.5/24.7 MB 3.5 MB/s eta 0:00:07
    --------------------------------------- 0.5/24.7 MB 2.8 MB/s eta 0:00:09
   - -------------------------------------- 0.6/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 0.8/24.7 MB 2.9 MB/s eta 0:00:09
   - ---------------


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: C:\Users\PH735WK\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
housing=fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [7]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)

In [8]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [9]:
data['Price']=housing.target

train, test split, hyperparameter tuning , mlflow

In [10]:
from urllib.parse import urlparse

#independenct and epenedent features

X=data.drop(columns=["Price"])
y=data['Price']

In [11]:
! pip install mlflow.models

ERROR: Could not find a version that satisfies the requirement mlflow.models (from versions: none)
ERROR: No matching distribution found for mlflow.models

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: C:\Users\PH735WK\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,
                             n_jobs=-1,verbose=True,scoring='neg_mean_squared_error')
    
    
    grid_search.fit(X_train,y_train)
    return grid_search

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

from mlflow.models import infer_signature
signature=infer_signature(X_train,y_train)

#Degine hyperparameter grid
param_grid={
    'n_estimators':[100,200],
    'max_depth':[5,10,None],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2]
}

#start the MLFLOW
with mlflow.start_run():
    grid_search=hyperparameter_tuning(X_train,y_train,param_grid)
    
    best_model=grid_search.best_estimator_
    
    y_pred=best_model.predict(X_test)
    
    mse=mean_squared_error(y_test,y_pred)
    
    ##Log best paarameters
    
    mlflow.log_param("Best_n_estimators",grid_search.best_params_['n_estimators'])
    mlflow.log_param("Best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_param("Best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("Best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)
    
    
    ##tracking url
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url_type_store!='file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name='Best random forest Model')
        
    else:
        mlflow.sklearn.log_model(best_model,"model",signature=signature)
    
    
    print(f"best hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared error: {mse}")    


Fitting 3 folds for each of 24 candidates, totalling 72 fits


KeyError: 'min_samples_Split'